In [1]:
import sys
sys.version

'3.7.2 (default, Jan 24 2019, 09:39:48) \n[Clang 10.0.0 (clang-1000.11.45.5)]'

In [4]:
!python -m pip install -qU apache-beam[gcp] tensorflow numpy Pillow

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os

import apache_beam as beam
import apache_beam.io.fileio as fileio
import tensorflow as tf

/Users/ponti/.pyenv/versions/3.7.2/Python.framework/Versions/3.7/lib/python3.7/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [2]:
tf.enable_eager_execution()

Define parameters and use View -> Cell Toolbar -> Tags to add `parameters` tag for [papermill](https://github.com/nteract/papermill)

In [3]:
# This cell is tagged `parameters`
input_directory = os.path.expanduser('~/Downloads/catvdog/input')
output_directory = os.path.expanduser('~/Downloads/catvdog/output')
dataset_name = 'catvdog'
num_shards = 2
has_labels = True
run_on_dataflow = False

In [4]:
if run_on_dataflow:
    runner = beam.runners.DataflowRunner()
else:
    runner = beam.runners.DirectRunner()

In [13]:
class TFExampleFromImageDoFn(beam.DoFn):
    def process(self, element):
        label, file_metadata = element

        def _bytes_feature(value):
            return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

        def _int_feature(value):
            return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

        def _float_feature(value):
            return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

        with tf.gfile.Open(file_metadata.path, 'rb') as f:
            img_bytes = f.read()
            img = tf.image.decode_jpeg(img_bytes, channels=3).numpy()

        width, height, _ = img.shape

        feature = {
            'image/bytes': _bytes_feature(img.tobytes()),
            'image/width': _int_feature(width),
            'image/height': _int_feature(height),
            'image/filename': _bytes_feature(file_metadata.path.encode())
        }

        if label is not None:
            feature['label'] = _bytes_feature(label.encode())

        example = tf.train.Example(features=tf.train.Features(feature=feature))

        yield example

In [ ]:
beam.io.filesystems.FileSystems.match()

In [17]:
def run_pipeline():
    path_expression = os.path.join(input_directory, '**/*.jpg')
    output_path = os.path.join(output_directory, dataset_name)

    with beam.Pipeline(runner=runner) as p: 
        (p
         | 'Create with path expression' >> beam.Create([path_expression])
         | 'Match Path Expression' >> fileio.MatchAll(fileio.EmptyMatchTreatment.DISALLOW)
         # The label is the parent folder of the file
         # Extract that folder name from the FileMetadata.path property
         | 'Extract labels' >> beam.Map(lambda f: (os.path.basename(os.path.dirname(f.path)), f))
         | 'To TF Example' >> beam.ParDo(TFExampleFromImageDoFn())
         | 'Save TF Records' >> beam.io.WriteToTFRecord(
             output_path, file_name_suffix='.tfrecord', num_shards=num_shards,
             shard_name_template='-SS', coder=beam.coders.ProtoCoder(tf.train.Example)
         ))

run_pipeline()

/Users/ponti/.pyenv/versions/3.7.2/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: MatchAll is experimental.
  del sys.path[0]
